In [ ]:
import base64
import io
import json
from PIL import Image
import tensorflow as tf

In [ ]:
img = Image.open('evaluation_images/bkk.jpg')
image_byte_arr = io.BytesIO()
img.save(image_byte_arr, "JPEG")
image_byte_arr = image_byte_arr.getvalue()

In [ ]:
import base64
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

credentials = GoogleCredentials.get_application_default()

# Build a representation of the Cloud ML API.
ml = discovery.build('ml', 'v1', credentials=credentials)

project = 'artful-reactor-182004'
model = 'StyleTransfer'
version = 'demov10'

name = 'projects/{}/models/{}/versions/{}'.format(project, model, version)

response = ml.projects().predict(
    name=name,
    body={'instances': {"image_byte": {"b64": base64.b64encode(image_byte_arr)}}}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])

In [ ]:
res = response['predictions'][0]['output_image']

sess = tf.InteractiveSession()
output_image =  tf.image.decode_jpeg(tf.decode_base64(res), channels=3).eval() 

In [ ]:
import matplotlib.pyplot as plt

_, sp = plt.subplots(figsize=(10, 10))

sp.imshow(output_image)
sp.set_title('Output image')

plt.show()